# Cell-Cell Communication Analysis with LIANA+

## Overview
This notebook performs cell-cell communication (CCC) analysis using LIANA+, a unified framework for ligand-receptor interaction inference.

### Objectives
1. Infer ligand-receptor interactions between cell types
2. Compare communication patterns in responders vs. non-responders
3. Identify resistance-associated signaling axes

### Why LIANA+?
- Aggregates multiple CCC methods (CellPhoneDB, NATMI, etc.)
- Consensus scoring reduces method-specific biases
- Integrates with scverse ecosystem

---

In [ ]:
import scanpy as sc
import liana as li
from liana.method import singlecellsignalr, connectome, cellphonedb, natmi, logfc, cellchat
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import yaml
import warnings

warnings.filterwarnings('ignore')

# Project paths
PROJECT_ROOT = Path("../..").resolve()
DATA_PROCESSED = PROJECT_ROOT / 'data' / 'processed' / 'scrna'
FIGURES = PROJECT_ROOT / 'results' / 'figures'
TABLES = PROJECT_ROOT / 'results' / 'tables'
CONFIG_PATH = PROJECT_ROOT / 'config' / 'analysis_params.yaml'

with open(CONFIG_PATH, 'r') as f:
    config = yaml.safe_load(f)

print(f"LIANA version: {li.__version__}")

## 1. Load Annotated Atlas

In [ ]:
# Load annotated data
adata = sc.read_h5ad(DATA_PROCESSED / 'integrated_atlas_annotated.h5ad')

print(f"Loaded atlas: {adata.n_obs} cells")
print(f"Cell types: {adata.obs['cell_type_major'].nunique()}")

# Use normalized log counts for LIANA
if 'normalized' in adata.layers:
    adata.X = adata.layers['normalized'].copy()

## 2. Run LIANA Analysis

In [ ]:
# Get LIANA parameters
liana_params = config['communication']['liana']

# Run LIANA with multiple methods
li.mt.rank_aggregate(
    adata,
    groupby='cell_type_major',
    resource_name=liana_params['resource'],
    expr_prop=config['communication']['min_expr_prop'],
    min_cells=10,
    verbose=True,
    use_raw=False
)

print("LIANA analysis complete!")

In [ ]:
# View results
liana_results = adata.uns['liana_res']

print(f"Total interactions: {len(liana_results)}")
print(f"\nTop 20 interactions by aggregate rank:")
display(liana_results.sort_values('magnitude_rank').head(20)[
    ['source', 'target', 'ligand_complex', 'receptor_complex', 'magnitude_rank', 'specificity_rank']
])

## 3. Visualize Communication Patterns

In [ ]:
# Dotplot of top interactions
li.pl.dotplot(
    adata,
    colour='magnitude_rank',
    size='specificity_rank',
    inverse_size=True,
    inverse_colour=True,
    source_labels=['Myeloid', 'DC', 'Fibroblast', 'Epithelial'],
    target_labels=['CD8_T', 'CD4_T', 'NK'],
    top_n=30,
    orderby='magnitude_rank',
    orderby_ascending=True,
    figure_size=(12, 8)
)

plt.savefig(FIGURES / 'liana_dotplot_immune.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Communication chord diagram
li.pl.circle_plot(
    adata,
    top_n=50,
    edge_color='magnitude_rank',
    inverse_colour=True,
    figure_size=(10, 10)
)

plt.savefig(FIGURES / 'liana_chord.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Identify Resistance-Associated Interactions

Compare CCC patterns between responders and non-responders.

In [ ]:
# If response labels are available:
# Run LIANA separately for responders and non-responders

def compare_ccc_by_response(adata):
    """
    Compare cell-cell communication between response groups.
    """
    if 'response' not in adata.obs.columns:
        print("Response labels not found. Skipping comparison.")
        return None
    
    results = {}
    
    for response in ['responder', 'non-responder']:
        adata_subset = adata[adata.obs['response'] == response].copy()
        
        if adata_subset.n_obs > 100:
            li.mt.rank_aggregate(
                adata_subset,
                groupby='cell_type_major',
                resource_name='consensus',
                verbose=False
            )
            
            results[response] = adata_subset.uns['liana_res']
    
    return results

# Uncomment when response labels are available:
# ccc_by_response = compare_ccc_by_response(adata)

print("Response comparison function defined")

## 5. Focus on Immune Checkpoints

In [ ]:
# Filter for checkpoint interactions
checkpoint_ligands = ['CD274', 'PDCD1LG2', 'CD80', 'CD86', 'LGALS9', 'NECTIN2', 'PVR']
checkpoint_receptors = ['PDCD1', 'CTLA4', 'HAVCR2', 'TIGIT', 'LAG3']

checkpoint_interactions = liana_results[
    (liana_results['ligand_complex'].str.contains('|'.join(checkpoint_ligands), na=False)) |
    (liana_results['receptor_complex'].str.contains('|'.join(checkpoint_receptors), na=False))
].sort_values('magnitude_rank')

print(f"Checkpoint interactions: {len(checkpoint_interactions)}")
display(checkpoint_interactions.head(20)[
    ['source', 'target', 'ligand_complex', 'receptor_complex', 'magnitude_rank']
])

In [ ]:
# Heatmap of checkpoint interactions
if len(checkpoint_interactions) > 0:
    # Pivot for heatmap
    pivot_data = checkpoint_interactions.pivot_table(
        index=['ligand_complex', 'receptor_complex'],
        columns='source',
        values='magnitude_rank',
        aggfunc='min'
    ).head(20)
    
    plt.figure(figsize=(12, 8))
    sns.heatmap(pivot_data, cmap='viridis_r', annot=True, fmt='.0f')
    plt.title('Immune Checkpoint Interactions (Magnitude Rank)')
    plt.tight_layout()
    plt.savefig(FIGURES / 'checkpoint_interactions_heatmap.png', dpi=150)
    plt.show()

## 6. Export Results

In [ ]:
# Save LIANA results
liana_results.to_csv(TABLES / 'liana_ccc_results.csv', index=False)
checkpoint_interactions.to_csv(TABLES / 'checkpoint_interactions.csv', index=False)

print(f"Results saved to {TABLES}")

## Summary

### Completed
- Inferred ligand-receptor interactions with LIANA+
- Visualized communication patterns
- Identified checkpoint interactions

### Key Findings
- Top communication axes between immune and stromal cells
- Checkpoint receptor engagement on T cells
- Potential resistance-driving interactions

### Next Steps
1. Spatial communication analysis in `07b_spatial_communication.ipynb`
2. NicheNet activity inference in `07c_nichenet_activity.ipynb`